In [1]:
from lxml import etree
import sqlite3

In [58]:
xml_db = 'wordnet/data/xml_entries.db'
sset_db = 'wordnet/data/all_synsets.db'
xml_rel_db = 'wordnet/data/xml_relations.db'

In [3]:
tree = etree.parse("estwn-et-2.0.0.beta.xml")

In [57]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def sqlTables(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS xml_relations(
                                        
                                        start_id TEXT NOT NULL,
                                        start_estwn TEXT NOT NULL,
                                        end_estwn TEXT NOT NULL,
                                        end_id TEXT NOT NULL,
                                        relType TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")

In [5]:

def fetch_data(tree):
    synsets = tree.xpath('/LexicalResource/Lexicon/LexicalEntry')
    j=0
    for sset in synsets:
        if len(sset)>1:
            for i in range(1, len(sset)):
                estwnId_list.append(sset[i].attrib['synset'])
                status_list.append(sset[i].attrib['status'])
                sset_list.append(sset[i].attrib['id'])
                pos_list.append(sset[0].attrib['partOfSpeech'])
                lemma_list.append(sset[0].attrib['writtenForm'])
        else:
            try:
                pos_list.append(sset[0].attrib['partOfSpeech'])
                lemma_list.append(sset[0].attrib['writtenForm'])
                word = sset[0].attrib['writtenForm']
                
                if " " in word:
                    var = word.replace(" ", "_")
                    word = var
                sset_list.append( "s-" + word + "-" + sset[0].attrib['partOfSpeech'] )
                
                estwnId_list.append(sset.attrib['id'])
                status_list.append("None")
            except:
                print("pass @",j)
                pass
        j+=1
        '''
        for relations
        
        if len(sset) > 2:
            for data in sset:
                #print(data.attrib)
                
                status_list.append(data.attrib['target'])
                conf_list.append((data.attrib['confidenceScore']))
                source_sense.append(data[0].attrib['sourceSense'])           
                '''

In [8]:
def upload_entries(db_file):
    sqlTables(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    
    with conn:
        i=0
        for i in range(len(sset_list)):
              
            cursor.execute("INSERT INTO XML_LexicalEntry(estwn_id,synset, writtenForm, pos, status) VALUES(?,?,?,?, ?)"\
                                                        ,(estwnId_list[i],sset_list[i],lemma_list[i], pos_list[i], status_list[i]))
            conn.commit()

In [65]:
def fetch_lemma(estwn_id):
    '''
    
    '''
    xml_conn = create_connection(xml_db)
    cursor = xml_conn.cursor()
    with xml_conn:
        #select all synsets with different indices from table.
        cursor.execute("SELECT DISTINCT writtenForm FROM XML_LexicalEntry WHERE estwn_id = ?", (estwn_id,))
        row = cursor.fetchone()
        #print(row[0])
        if row is not None:
            return row[0]

In [66]:
def fetch_ssetId(estwn_id):
    
    lemma = fetch_lemma(estwn_id)
    
    db_conn = create_connection(sset_db)
    cursor = db_conn.cursor()
    with db_conn:
        cursor.execute("SELECT DISTINCT synset_id FROM synset_table WHERE synset_word = ?", (lemma,))
        row = cursor.fetchone()
        if row is not None:
            #print(row[0])
            return row[0]

In [85]:
def upload_synsets(db_file):
        
    end_estId=0
    end_id=0
    rel=0
    start_estId=0
    start_id=0
    
    sqlTables(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    ssets = tree.xpath('/LexicalResource/Lexicon/Synset')    
    with conn:
        for i in range(len(ssets)):
            if len(ssets[i]) > 1:
                print("if")
                start_estId = ssets[i].attrib['id']
                start_id = fetch_ssetId(start_estId)
                for sset in ssets:
                    try:
                        rel = sset.attrib['relType']
                        end_estId = sset.attrib['target']
                    except:
                        pass
                    #for start ID find written form from lexical db WHERE start_estId == estwn_id,
                    #get writtenForm of that estwn_id
                    #from synset_db get DISTINCT synset_id WHERE writtenForm == synset_word
    
                    #for end ID find writtenForm from lexical db WHERE end_estId == estwn_id,
                    #get writtenForm of that estwn_id
                    #from sset_db get DISTINCT synset_id WHERE writtenForm == synset_word
                    end_id = fetch_ssetId(end_estId)
                    print(end_estId)
                    print(rel)
                    print(end_id)
            else:
                print("else")
                start_estId = ssets[i].attrib['id']
                start_id = fetch_ssetId(start_estId)
                try:
                    rel = sset.attrib['relType']
                    end_estId = sset.attrib['target']
                except:
                    continue
            print(start_id)
            cursor.execute("INSERT INTO xml_relations(start_id, start_estwn, end_estwn, end_id, relType) VALUES(?,?,?,?,?)"\
                                                            ,(start_id, start_estId, end_estId, end_id,rel))
            conn.commit()
            
                

In [7]:
estwnId_list = []
status_list=[]
pos_list = []
lemma_list = []
sset_list=[]


fetch_data(tree)

In [79]:
print(len(estwnId_list))
print(len(status_list))
print(len(pos_list))
print(len(lemma_list))
print(len(sset_list))

upload_entries(xml_db)

133161
133161
133161
133161
133161


KeyboardInterrupt: 

Get all lexical entries

In [86]:
upload_synsets(xml_rel_db)

if
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
None
0
0
Non

KeyboardInterrupt: 

In [18]:
entries = tree.xpath('/LexicalResource/Lexicon/LexicalEntry')
print(len(entries))

113840


In [68]:
var=entries[784]
print(var.attrib)
print(len(var))

{'id': 'w526555'}
1


In [79]:
entry = entries[8661]
print(entry[0].attrib)
print(len(entry))
entry[0].attrib['writtenForm']
entry.attrib['id']

{'writtenForm': ' hallikaspunane', 'partOfSpeech': 'a'}
1


'w234083'

Each lexicon element is al lemma and pos pair and it can correspond to many synsets

In [50]:
entry = entries[6]
print(entry[0].tag)
print(entry[0].attrib['writtenForm'])
print(entry[0].attrib['partOfSpeech'])
print(entry[1].attrib['status'])
entry[1].attrib['id']

Lemma
väntaja
n
unchecked


's-väntaja-n3'

In [91]:
subset = entries[104]
print(len(subset))
print(subset[0].tag)
print(subset[0].attrib['writtenForm'])
print(subset[0].attrib)

7
Lemma
kaotaja
{'partOfSpeech': 'n', 'writtenForm': 'kaotaja'}


Corresponding synset mapping is defined as sense element in the second, third,... components of an entry

In [43]:
print(entry[0].tag)
print('Long name = ', entry[1].attrib['id'])
print('Status = ', entry[1].attrib['status'])
print('synset =', entry[1].attrib['synset'])


Lemma
Long name =  s-röntgen-n1
Status =  unchecked
synset = estwn-et-41417-n


In [ ]:
There can be several sense components

In [65]:
print(len(entries[2]))
print(entries[2][1].attrib)
print(entries[2][2].attrib)

3
{'synset': 'estwn-et-3972-v', 'id': 's-sisse_ajama-v1', 'status': 'unchecked'}
{'synset': 'estwn-et-19302-v', 'id': 's-sisse_ajama-v2', 'status': 'unchecked'}


Get all sysets

In [45]:
synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
print(len(synsets))

83557


In [44]:
sset = synsets[10]
print(len(sset))
print(sset.attrib)

NameError: name 'synsets' is not defined

The most important property of a synset is its identifier, status and confidence 

In [13]:
synset=synsets[0]
print('synset_id=', synset.attrib['id'])
print('synset_status=', synset.attrib['status'])
print('confidence_score=', synset.attrib['confidenceScore'])


synset_id= estwn-et-8645-n
synset_status= checked
confidence_score= 1.0


The first element of the synset chhild is its definition that links back to lexicon element senses

In [63]:
print(synset[0].tag)
print('language = ', synset[0].attrib['language'])
print('lecicon_sense = ', synset[0].attrib['sourceSense'])

Definition
language =  et
lecicon_sense =  s-kalapüük-n1


Next elements are relations between synsets spacified through target and relation type

In [21]:
synset = synsets[6]

In [24]:

print(synset[1].tag)
print('rel_confidence=', synset[1].attrib['confidenceScore'])
print('rel_type=', synset[1].attrib['relType'])
print('rel_status=', synset[1].attrib['status'])
print('rel_target=', synset[1].attrib['target'])
print('synset_id=', synset.attrib['id'])


SynsetRelation
rel_confidence= 1.0
rel_type= hypernym
rel_status= unchecked
rel_target= estwn-et-34850-n
synset_id= estwn-et-35939-n


You can access these elements by specifying relative paths

In [160]:
definition = synset.xpath('Definition')
relations = synset.xpath('SynsetRelation')
print(len(definition),len(relations))

1 105


In [44]:
print(len(synsets))
print(len(entries))

NameError: name 'synsets' is not defined

In [90]:
print(entry[1].attrib)
print(synset[100].attrib)

{'synset': 'estwn-et-41417-n', 'id': 's-röntgen-n1', 'status': 'unchecked'}
{'target': 'estwn-et-71125-n', 'relType': 'hyponym', 'status': 'unchecked', 'confidenceScore': '1.0'}
